<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/master/landlab_header.png"></a>

# Modeling Hillslopes and Channels with Landlab
The original version of this exercise was donated by Andy Wickert at the University of Minnesota.

<hr>
<small> For instructions on how to run an interactive IPython notebook, click here: <a href="https://github.com/landlab/tutorials/blob/master/README.md">https://github.com/landlab/tutorials/blob/master/README.md</a></small><br>
<small>For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

** What is this notebook? **

This notebook illustrates a landscape evolution model in which the landscape evolves according to the equation:

\begin{equation}
 \frac{\partial z}{\partial t} = -K_\text{sp} A^{m_{sp}} S^{n_{sp}} + K_\text{hs} \frac{\partial^2 z}{\partial x^2} + U
\end{equation}
Here, $K$ are coefficients on the fluvial (_sp_) and hillslope (_hs_) parts of the equation, and $m_{sp}$ and $n_{sp}$ are positive exponents, usually thought to have a ratio, $m_{sp}/n_{sp} \approx 0.5$. $A$ is drainage area and $S$ is the slope of steepest descent ($-\frac{dz}{dx}$) where $x$ is horizontal distance (positive in the downslope direction) and $z$ is elevation. (If slope is negative there is no fluvial erosion.) $U$ is an externally-applied uplift field.

The first term on the right hand side of the equation is the fluvial erosion term, which is also known as the stream power equation. The second term on the right hand side of the equation is elevation changes via linear diffusion, and linear diffusion is one way in which to describe hillslope sediment transport.

** What will you do? **

In this exercise you will modify the code to get a better understanding of how different parameters control landscape evolution and form.

Start by sequentially running each code block without changing anything. At the end of the notebook you will see the questions that you need to answer by changing parts of the code and rerunning it. 

Remember that you can always go to the _Kernel_ pulldown menu at the top and choose _Restart & Clear Output_ or _Restart & Run All_ if you change things and want to start afresh. If you just change one code block and rerun only that code block, only the parts of the code in that code block will be updated. (E.g. if you change parameters but don't reset the code blocks that initialize run time or topography, then these values will not be reset.)

** Now on to the code... **

First we have to import the parts of Python and Landlab that are needed to run this code. You should not have to change this first code block.

In [ ]:
# Code Block 1
import numpy as np
from landlab import RasterModelGrid, HexModelGrid
from landlab.components import StreamPowerEroder, FlowRouter, \
     LinearDiffuser
from landlab import imshow_grid
from copy import deepcopy
from matplotlib import pyplot as plt
#below is to make plots show up in the notebook
%matplotlib inline 

** Now we set parameters **

This part you will need to change for the different exercises. 

Note that Landlab does not impose units, but it assumes that all units are consistent. We will assume that everything is given in _meters_ (m) and _years_ (yr).

In [ ]:
# Code Block 2
uplift_rate = 0.001  # [m/yr]
K_sp = 1.e-5  # units vary depending on m_sp and n_sp
m_sp = 0.5 # exponent on drainage area in stream power equation
n_sp = 1. # exponent on slope in stream power equation
K_hs = 0.1  # [m^2/yr]
tmax = 3E5  # time for the model to run [yr]
ncells_side = 150 # number of raster cells on each side
dxy  = 50 # side length of a raster model cell, or resolution [m]

** Make a grid. **

This part you may want to change.

In [ ]:
# Code Block 3
# Below is a raster (square cells) grid, with equal width and height 
mg = RasterModelGrid((ncells_side, ncells_side), dxy)

# Below is a Hexagonal grid (hexagonal cells, 
# with equal width and height (a square)
#mg = HexModelGrid(ncells_side, ncells_side, dxy, shape='rect')

# Below is a hexagonal grid (hexagonal cells), 
# in which the grid is also a hexagon
#mg = HexModelGrid(ncells_side, ncells_side/2, dxy, shape='hex')

Set some variables related to time. 

In [ ]:
# Code Block 4
dt = 50000 # time step [yr]
total_time = 0 # amount of time the landscape has evolved [yr]

t = np.arange(0, tmax, dt) # each of the time steps that the code will run

Here we make the initial grid of elevation. 

In [ ]:
# Code Block 5
np.random.seed(0) # seed set to zero so our figures are reproducible
mg_noise = np.random.rand(mg.number_of_nodes)/1000. # intial noise on elevation gri

# set up the elevation on the grid
zr = mg.add_zeros('node', 'topographic__elevation')
zr += mg_noise

Intializing all of the process components that do the work.

In [ ]:
# Code Block 6
frr = FlowRouter(mg) # intializing flow routing
spr = StreamPowerEroder(mg, K_sp=K_sp, m_sp=m_sp, n_sp=n_sp, threshold_sp=0,
                        use_Q=None) #initializing stream power incision
dfn = LinearDiffuser(mg, linear_diffusivity=K_hs, deposit = False) # initializing linear diffusion 

Now for the code loop. Note that you can rerun this code block many times, and as long as you don't rerun any of the code boxes above, it will take the already evolved landscape and evolve it even more.

In [ ]:
# Code Block 7
for ti in t:
    zr[mg.core_nodes] += uplift_rate*dt # uplift the landscape
    dfn.run_one_step(dt) # diffuse the landscape
    frr.run_one_step() # route flow
    spr.run_one_step(dt) # fluvial incision
    total_time += dt # update time keeper
    print total_time

Plot the topography.

In [ ]:
# Code Block 8
imshow_grid(mg, 'topographic__elevation', grid_units=('m', 'm'),
                var_name='Elevation (m)')
title_text = '$K_{sp}$='+str(K_sp) + '; $K_{hs}$='+str(K_hs) + '; $time$='+str(total_time) + '; $dx$='+str(dxy)
plt.title(title_text)

Plot the slope and area data at each point (in log-log space).

In [ ]:
# Code Block 9
edge = np.unique(mg.neighbors_at_node[mg.boundary_nodes, :])
not_edge = np.in1d(mg.nodes.flatten(), edge, assume_unique=True,
                       invert=True)
plt.loglog(mg.at_node['drainage_area'][not_edge],
           mg.at_node['topographic__steepest_slope'][not_edge], 'x')
#xlim([1.e3, 1.e7])
plt.ylabel('Topographic slope')
plt.xlabel('Drainage area (m^2)')
plt.title(title_text)

Answer the following questions using the code above. All solutions should be typed, and supporting figures (produced using the code) should be embedded in your final document. (Download or screenshoot the figures.) You may want to make other plots with the data you collect using this model. You are free to make those plots using whatever software you choose.

In parts of this exercise you need to work with your classmates. You are encouraged to discuss how to use the model and model results with your classmates, however the write-up that you hand in must be your own work.

1. **Hillslope vs. Fluvial Processes. ** Using the parameters provided in the initial notebook, run the landscape to steady state, or the point at which the topography and the slope-area relationship stop changing (i.e. erosion equals rock uplift). (You can keep rerunning Code Block 7 until steady state is reached.) Use the plots of slope and area to estimate where the hillslope–fluvial transition is (or in otherwords, the threshold drainage area for channel heads). Now try keeping $K_{sp}$ the same but increase and decrease $K_{hs}$ (in Code Block 2). How does the transition from hillslope to channel change with changes in $K_{hs}$? Now try keeping $K_{hs}$ the same but increase and decrease $K_{sp}$ (in Code Block 2). How does the transition from hillslope to channel change with changes in $K_{sp}$? Produce a relationship between the different values of $K_{sp}$, $K_{hs}$ and the threshold drainage area. Remember to run all of your different parameter values to steady state before estimating the threshold drainage area.

2. **Uplift and erosion. ** Now, perform a similar set of exercises as you did in exercise 1, but also systematically vary uplift rate. Work in teams, and each person should choose two combinations of $K_{sp}$ and $K_{hs}$ and three uplift rates (for a total of 6 runs). Make sure the values that you choose do not overlap with your group members. This time make sure you document the transition from hillslope to fluvial process, and you also note the maximum steady state elevation for each combination of uplift, $K_{sp}$ and $K_{hs}$. Produce relationships to show how the threshold and maximum elevation change with the different variables.

3. ** Landscapes evolving to steady state. ** Try changing the grid size. Keep running the model until it reaches steady state. Keep record of this time. Now trying changing the rock uplift rate and record the time to steady state. Change the resolution (i.e. the cell size, Code Block 2) and record the time to steady state. Combine in groups to gather enough model outputs to form relationships between time to steady state and grid size, resolution, or  rock uplift rate. After doing this, choose a constant model set-up (fix grid size, resolution, and uplift rate) and run the model from a flat initial condition with noise to steady state, but capture snapshots of the topography and slope-area relationship as the landscape evolves. Now try changing one of the variables (grid size, resolution, or rock uplift rate) and do the same thing. Note any qualitative or quantitative differences you see (or do not see) between these two different evolutionary sequences.  

4. ** Free-form exploration. ** Try changing the grid type (Code Block 3), grid size (Code Block 2), stream power exponents (Code Block 2), distribution of uplift rate (e.g., what happens if you have just part of the landscape experience uplift, a bit trickier, as uplift in Code Block 2 will need to be changed to an array), etc. Based on what you observe, create a consistent geomorphic history of the system. Creativity is expected here!